In [1]:
import torch
import numpy as np
import pyLDAvis
import pandas as pd
import itertools
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [9]:
import os
# os.chdir("Documents/GitHub/Topic-Mapping")
print(os.getcwd())

/Users/carterward/Documents/GitHub/Topic-Mapping
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)


# restore np.load for future normal usage
# np.load = np_load_old

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
def softmax(x):
    # x has shape [batch_size, n_classes]
    e = np.exp(x)
    n = np.sum(e, 1, keepdims=True)
    return e/n

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
path_to_dir = "models/articles/"
n_topics = 25

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
dataset = pd.read_csv("data/large_article_sample.csv")
# "integer -> word" decoder 
decoder = np.load(path_to_dir+'decoder.npy')[()]

vocab = list(decoder.values())

# for restoring document ids, "id used while training -> initial id"
doc_decoder = np.load(path_to_dir+'doc_decoder.npy')[()]
docs = dataset.iloc[list(doc_decoder.values())]["tokenized_content"].values

doc_lengths = np.array([0 if pd.isna(tok) else len(tok.split()) for tok in docs])

docs_compiled = list(itertools.chain.from_iterable([doc.split() for doc in docs if not pd.isna(doc)]))
term_frequency = np.array([docs_compiled.count(word) for word in vocab])

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/asyncio/events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


In [13]:
state = torch.load(path_to_dir+'lda2vec_models/40_epoch_model_state.pytorch', map_location=lambda storage, loc: storage)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
doc_weights = state['doc_weights.weight'].cpu().clone().numpy()
topic_vectors = state['topics.topic_vectors'].cpu().clone().numpy()
resulted_word_vectors = state['neg.embedding.weight'].cpu().clone().numpy()

# distribution over the topics for each document
topic_dist = softmax(doc_weights)

# vector representation of the documents
doc_vecs = np.matmul(topic_dist, topic_vectors)
# rows are topics, cols are vocab, vals are probabilities
topic_term_dist = softmax(np.matmul(topic_vectors, resulted_word_vectors.T))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
print(doc_vecs.shape)
print(topic_dist.shape)
print(topic_vectors.shape)
print(resulted_word_vectors.shape)
print(doc_weights.shape)
print(topic_term_dist.shape)

(9648, 50)
(9648, 25)
(25, 50)
(12402, 50)
(9648, 25)
(25, 12402)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
dashboard = pyLDAvis.prepare(topic_term_dist, topic_dist, doc_lengths, vocab, term_frequency)
pyLDAvis.display(dashboard)
pyLDAvis.save_html(dashboard, "plots/articles_lda2vec.html")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
similarity = np.matmul(topic_vectors, resulted_word_vectors.T)
most_similar = similarity.argsort(axis=1)[:, -10:]
topic_matrix = []
for j in range(n_topics):
    topic_matrix.append([decoder[i] for i in reversed(most_similar[j])])
    topic_words = ' '.join([decoder[i] for i in reversed(most_similar[j])])
    print('topic', j + 1, ':', topic_words)

topic 1 : isis syria islamic iraq syrian militant mosul rebel troop aleppo
topic 2 : parenthood abortion medicaid planned funding prescription budget repeal legislation opioid
topic 3 : castro airline fidel flight munoz passenger airlines lewandowski mets dao
topic 4 : ryan speaker paul climate schlafly carbon mcconnell gravitational astronomer telescope
topic 5 : hogan pruitt epa gas gawker energy denton oklahoma daulerio oil
topic 6 : pipeline college ms. sioux teacher parent graduate dakota standing education
topic 7 : nfl player giants korea coach korean kim quarterback baseball broncos
topic 8 : song music album fashion band designer singer sing dress dance
topic 9 : gorsuch spicer mrs. conway bannon nsc grassley kissinger partisan nixon
topic 10 : gun shooting shoot ailes dallas fox cop o’reilly castile handgun
topic 11 : google apple app user tesla iphone internet technology software amazon
topic 12 : fbi comey intelligence server flynn classified nunes leak probe hack
topic 13 

In [18]:
d = corpora.Dictionary()
d.id2token = decoder
d.token2id = {value: key for key, value in decoder.items()}
docs = [doc for doc in docs if not pd.isna(doc)]
docs = [doc.split() for doc in docs]

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/asyncio/events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


In [19]:
cm = CoherenceModel(topics=topic_matrix, texts=docs, dictionary=d, coherence='c_v')
cm.get_coherence()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.5902794107028866